In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import lightning.pytorch as pl
import torch
import wandb
from sdofm import utils
from sdofm.datasets import SDOMLDataModule, DegradedSDOMLDataModule
from sdofm.pretraining import MAE, SAMAE
from sdofm.finetuning import Autocalibration

In [4]:
import omegaconf

cfg = omegaconf.OmegaConf.load("../experiments/finetune_32.2M_mae_autocalibration.yaml")

In [4]:
# from sdofm.utils import flatten_dict
# import yaml

# data = flatten_dict(cfg, sep="___")
# with open('testingout.yaml', 'w+') as outfile:
#     yaml.dump(data, outfile, default_flow_style=False)
# outfile.close()

In [5]:
# from omegaconf import OmegaConf
# OmegaConf.save(config, "testout.yaml")

In [6]:
degraded_data_module = DegradedSDOMLDataModule(
    hmi_path=None,
    aia_path=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.aia
    ),
    eve_path=None,
    components=cfg.data.sdoml.components,
    wavelengths=cfg.data.sdoml.wavelengths,
    ions=cfg.data.sdoml.ions,
    frequency=cfg.data.sdoml.frequency,
    batch_size=cfg.model.opt.batch_size,
    num_workers=cfg.data.num_workers,
    val_months=cfg.data.month_splits.val,
    test_months=cfg.data.month_splits.test,
    holdout_months=cfg.data.month_splits.holdout,
    cache_dir=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.cache
    ),
    min_date=cfg.data.min_date,
    max_date=cfg.data.max_date,
    num_frames=1,
)
degraded_data_module.setup()

[* CACHE SYSTEM *] Found cached index data in /mnt/sdoml/cache/aligndata_AIA_FULL_12min.csv.
[* CACHE SYSTEM *] Found cached normalization data in /mnt/sdoml/cache/normalizations_AIA_FULL_12min.json.
[* CACHE SYSTEM *] Found cached HMI mask data in /mnt/sdoml/cache/hmi_mask_512x512.npy.


In [7]:
# import yaml
# from pathlib import Path

# conf = yaml.safe_load(
#     Path(
#         "/home/walsh/repos/SDO-FM/output/2024-04-17-00-41-29/0/wandb/latest-run/files/config.yaml"
#     ).read_text()
# )
# backbone_cfg = utils.unflatten_dict(conf, sep="___", wandb_mode=True)
# backbone_cfg

In [8]:
# backbone_cfg.model.opt

In [9]:
backbone_cfg=cfg
backbone = MAE.load_from_checkpoint(
    **backbone_cfg.model.mae,
    optimiser=backbone_cfg.model.opt.optimiser,
    lr=backbone_cfg.model.opt.learning_rate,
    weight_decay=backbone_cfg.model.opt.weight_decay,
    checkpoint_path="/home/walsh/SDO-FM/outputs/2024-05-24/18-35-29/sdofm/chvgwzkk/checkpoints/epoch=3-step=176.ckpt"
)

In [10]:
model = Autocalibration(
    **cfg.model.mae,
    **cfg.model.dimming,
    optimiser=cfg.model.opt.optimiser,
    lr=cfg.model.opt.learning_rate,
    weight_decay=cfg.model.opt.weight_decay,
    backbone=backbone,
    hyperparam_ignore=['backbone']
)

In [11]:
trainer = pl.Trainer(
    devices=1, accelerator=cfg.experiment.accelerator, max_epochs=cfg.model.opt.epochs
)
trainer.fit(model=model, datamodule=degraded_data_module)

I0000 00:00:1716576708.615099 2187280 pjrt_api.cc:100] GetPjrtApi was found for tpu at /usr/local/lib/python3.10/dist-packages/libtpu/libtpu.so
I0000 00:00:1716576708.615168 2187280 pjrt_api.cc:79] PJRT_Api is set for device type tpu
I0000 00:00:1716576708.615199 2187280 pjrt_api.cc:146] The PJRT plugin has PJRT API version 0.46. The framework PJRT API version is 0.46.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185148.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185148.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185148.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185148.2187280': Permission denied
Could not ope

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.WARNING.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/t

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not open any log file.
Could not open the log file '/tmp/tpu_logs/tpu_driver.t1v-n-1dd81944-w-0.walsh.log.INFO.20240524-185227.2187280': Permission denied
Could not ope

AttributeError: 'MAE' object has no attribute 'forward_encoder'

: 